<a href="https://colab.research.google.com/github/sugiyama404/ReinfoceLearningForTrading/blob/main/Categorical_DQN/categorical_dqn_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
from google.colab import drive
import copy

from datetime import datetime
from matplotlib import pyplot as plt
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ReLU, Softmax, Reshape
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.losses import Huber, CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_crossentropy

from tensorflow.keras.utils import Progbar
from sklearn.preprocessing import StandardScaler

mode = 'train'
name = 'c51'

level = 1
if level == 2:
    name += name + 'lv2'

drive.mount('/content/drive/')
nov_dir = 'Colab Notebooks/dataset/reinforcement_learning/'
nov_path = '/content/drive/My Drive/' + nov_dir + f'sp500_{mode}.csv'

exp_dir = 'Colab Notebooks/workspace/export/'
mdl_dir = '/content/drive/My Drive/' + exp_dir + 'models'
csv_path = '/content/drive/My Drive/' + exp_dir + f'csv_data/{name}_{mode}.csv'

df = pd.read_csv(nov_path)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
class Environment:
    def __init__(self, df, initial_money=100000, mode = 'test', commission = 0):

        self.df = df.dropna().reset_index()

        self.df_total_steps  = len(self.df)-1
        self.initial_money   = initial_money
        self.mode            = mode
        self.commission      = commission
        self.trade_time      = None
        self.trade_win       = None
        self.brfore_buy_cash = None
        self.action_space    = np.array([0, 1, 2]) # buy,hold,sell
        self.hold_a_position = None
        self.now_price       = None
        self.cash_in_hand    = None
        self.sell_price      = None
        self.buy_price       = None

        self.reset()
        
    def reset(self):

        self.trade_time      = 0
        self.trade_win       = 0
        self.brfore_buy_cash = 0
        self.end_step        = self.df_total_steps
        self.now_step        = 0
        self.hold_a_position = 0.0
        self.now_price       = self.df.loc[self.now_step, 'SP500']
        self.cash_in_hand    = self.initial_money
        self.sell_price      = 0
        self.buy_price       = 0

        return self._get_now_state()

    def step(self, action):

        self.now_step += 1
        self.now_price = self.df.loc[self.now_step, 'SP500']
 
        done = (self.end_step == self.now_step)

        self.sell_price = 0
        self._trade(action,done)
        reward = 0
        if (self.sell_price > 0) and (self.buy_price > 0) and ((self.sell_price - self.buy_price) != 0):
            reward = (self.sell_price - self.buy_price) / self.buy_price
            self.buy_price = 0
        cur_revenue = self._get_revenue()
 
        info = { 'cur_revenue' : cur_revenue , 'trade_time' : self.trade_time, 'trade_win' : self.trade_win }

        return self._get_now_state(), reward, done, info

    def _get_now_state(self):
        state = np.empty(3)
        state[0] = self.hold_a_position
        state[1] = self.now_price
        state[2] = self.cash_in_hand
        return state

    def _get_revenue(self): 
        return self.hold_a_position * self.now_price + self.cash_in_hand

    def _trade(self, action,lastorder = False):
        if lastorder:
            if self.hold_a_position != 0:
                self.cash_in_hand += self.now_price * self.hold_a_position
                self.hold_a_position = 0
                self.trade_time += 1
                if self.cash_in_hand > self.brfore_buy_cash:
                    self.trade_win += 1
        else:
            if self.action_space[0] == action: # buy
                if self.hold_a_position == 0:
                    buy_flag = True
                    self.brfore_buy_cash = copy.copy(self.cash_in_hand)
                    while buy_flag:
                        if self.cash_in_hand > self.now_price:
                            self.hold_a_position += 1
                            self.buy_price += self.now_price
                            self.cash_in_hand -= self.now_price + self.commission * self.now_price
                        else:
                            buy_flag = False
            if self.action_space[2] == action: # sell
                if self.hold_a_position != 0:
                    self.sell_price += self.now_price * self.hold_a_position
                    self.cash_in_hand += self.now_price * self.hold_a_position - self.commission * self.now_price * self.hold_a_position
                    self.hold_a_position = 0
                    self.trade_time += 1
                    if self.cash_in_hand > self.brfore_buy_cash:
                        self.trade_win += 1

In [ ]:
class ReplayMemory:
    def __init__(self, max_size=500, batch_size=32):

        self.cntr = 0
        self.size = 0
        self.max_size = max_size
        self.batch_size = batch_size
        self.states_memory = np.zeros([self.max_size, 3], dtype=np.float32)
        self.next_states_memory = np.zeros([self.max_size, 3], dtype=np.float32)
        self.acts_memory = np.zeros(self.max_size, dtype=np.uint8)
        self.rewards_memory = np.zeros(self.max_size, dtype=np.float32)
        self.done_memory = np.zeros(self.max_size, dtype=np.uint8)

    def store_transition(self, state, act, reward, next_state, done):
        self.states_memory[self.cntr] = state
        self.next_states_memory[self.cntr] = next_state
        self.acts_memory[self.cntr] = act
        self.rewards_memory[self.cntr] = reward
        self.done_memory[self.cntr] = done
        self.cntr = (self.cntr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def random_sampling(self):
        mb_index = np.random.choice(self.size, self.batch_size, replace=False)
        key = ['state','next_state','act','reward','done']
        value = [self.states_memory[mb_index],self.next_states_memory[mb_index],
                 self.acts_memory[mb_index],self.rewards_memory[mb_index],
                 self.done_memory[mb_index]]
        dict1=dict(zip(key,value))

        return dict1

In [ ]:
class Brain:
    def __init__(self):

        self.action_space = 3
        self.n_atoms = 51

        learning_rate = 0.00001
        neurons_per_layer = 24
        optimizer = Adam(learning_rate=learning_rate, epsilon=0.001)

        model = Sequential()
        model.add(Dense(neurons_per_layer * 2, input_shape=(3,)))
        model.add(ReLU()) 
        model.add(Dense(neurons_per_layer * 4))
        model.add(ReLU())
        model.add(Dense(self.action_space * self.n_atoms))
        model.add(Reshape((self.action_space, self.n_atoms)))
        model.add(Softmax())
        model.compile(loss=Huber(), optimizer=optimizer)
        model.summary()
        self.model = model

In [ ]:
class Agent(Brain, ReplayMemory):
    def __init__(self, max_size=500, batch_size=32):
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.r = 0.99995
        self.batch_size = batch_size
        self.n_atoms = 51
        self.action_space = 3
        self.Vmin = -10
        self.Vmax = 10
        self.delta_z = (self.Vmax - self.Vmin) / (self.n_atoms - 1)
        self.Z = np.linspace(self.Vmin, self.Vmax, self.n_atoms)
        Brain.__init__(self)
        ReplayMemory.__init__(self, max_size, batch_size)

    def update_replay_memory(self, state, action, reward, next_state, done):
        self.store_transition(state, action, reward, next_state, done)

    def act(self, state):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.r

        if np.random.rand() <= self.epsilon:
            return np.random.choice(3)

        selected_actions, _ = self._predict(state)
        return selected_actions[0][0].numpy()

    def _predict(self, state):
        probs = self.model.predict(state)
        q_means = tf.reduce_sum(probs * self.Z, axis=2, keepdims=True) # q_means:  (32, 3, 1)
        selected_actions = tf.argmax(q_means, axis=1) # selected_actions: (32, 1)
        return selected_actions, probs

    def _actions_onehot(self, actions):
        actions = tf.cast(actions, tf.int32)
        actions_onehot = tf.one_hot(actions, self.action_space, axis=1)
        condition = tf.equal(actions_onehot, 1)
        case_true = tf.ones(self.n_atoms, dtype=tf.dtypes.float32)
        case_false = tf.zeros(self.n_atoms, dtype=tf.dtypes.float32)
        mask = tf.where(condition, case_true, case_false)
        return mask

    def _bellman_equation(self, rewards, dones, next_z):

        target_zs = np.zeros((self.batch_size, self.n_atoms))

        for j in range(self.n_atoms):

            tZ_j = np.minimum(self.Vmax, np.maximum(self.Vmin, rewards + self.gamma * self.Z[j]))
            bj = (tZ_j - self.Vmin) / self.delta_z

            lower_bj = np.floor(bj).astype(np.int32)
            upper_bj = np.ceil(bj).astype(np.int32)

            eq_mask = (lower_bj == upper_bj)
            neq_mask = (lower_bj != upper_bj)

            lower_probs = 1 - (bj - lower_bj)
            upper_probs = 1 - (upper_bj - bj)

            next_z = next_z[:, [j]]  # next_z.shape:  (32, 51) # next_z.shape:  (32, 1)
            next_z = np.reshape(next_z, [self.batch_size, ])
            indices = np.arange(self.batch_size).reshape(-1, 1)
            target_zs[indices[neq_mask], lower_bj[neq_mask]] += (lower_probs * next_z)[neq_mask] # lower_probs: (32,) neq_mask: (32,)
            target_zs[indices[neq_mask], upper_bj[neq_mask]] += (upper_probs * next_z)[neq_mask]

            target_zs[indices[eq_mask], lower_bj[eq_mask]] += (0.5 * next_z)[eq_mask]
            target_zs[indices[eq_mask], upper_bj[eq_mask]] += (0.5 * next_z)[eq_mask]

        for batch_idx in range(self.batch_size):

            if not dones[batch_idx]:
                continue
            else:
                target_zs[batch_idx, :] = 0
                tZ = np.minimum(self.Vmax, np.maximum(self.Vmin, rewards[batch_idx]))
                bj = (tZ - self.Vmin) / self.delta_z

                lower_bj = np.floor(bj).astype(np.int32)
                upper_bj = np.ceil(bj).astype(np.int32)

                if lower_bj == upper_bj:
                    target_zs[batch_idx, lower_bj] += 1.0
                else:
                    target_zs[batch_idx, lower_bj] += 1 - (bj - lower_bj)
                    target_zs[batch_idx, upper_bj] += 1 - (upper_bj - bj)

        return target_zs


    def replay(self):
        if self.size < self.batch_size:
            return

        m_batch = self.random_sampling()
        states, next_states, actions, rewards, dones = m_batch['state'], m_batch['next_state'], m_batch['act'], m_batch['reward'], m_batch['done']

        next_actions, next_probs = self._predict(next_states)

        onehot_next_action = self._actions_onehot(next_actions)
        next_z = tf.reduce_sum(next_probs * onehot_next_action, axis=1).numpy()
        target_z = self._bellman_equation(rewards, dones, next_z)

        actions = np.reshape(actions, [self.batch_size, 1])
        onehot_action = self._actions_onehot(actions)

        with tf.GradientTape() as tape:
            probs = self.model(states, training=True)
            z = tf.reduce_sum(probs * onehot_action, axis=1)
            z = tf.clip_by_value(z, 1e-4, 1.0)

            loss = categorical_crossentropy(z, target_z)
            loss = tf.reduce_mean(loss)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
class Main:
    def __init__(self, env, agent, mdl_dir, name, episodes_times = 200, mode = 'test'):
        self.env            = env
        self.agent          = agent
        self.mdl_dir        = mdl_dir
        self.scaler         = self._standard_scaler(self.env)
        self.episodes_times = episodes_times
        self.mode           = mode
        self.name           = name

        with open(csv_path, 'w') as f:
            row = 'FixedProfit,TradeTimes,TradeWin'
            print(row, file=f)

        if self.mode == 'test':
            self._load()
            self.agent.epsilon = 0.01

    def play_game(self):
        for episode in range(self.episodes_times):

            if (episode % 10 == 0):
                metrics_names = ['FixedProfit','TradeTimes','TradeWin']
                pb_i = Progbar(10, stateful_metrics=metrics_names)
                p_mean,trade_time,win_time = np.array([]),np.array([]),np.array([])

            state = self.env.reset()
            state = self.scaler.transform([state])
            done  = False
        
            while not done:
                action = self.agent.act(state)
                next_state, reward, done, info = self.env.step(action)
                next_state = self.scaler.transform([next_state])
                reward = self._reward_clipping(reward)

                if self.mode == 'train':
                    self.agent.update_replay_memory(state, action, reward, next_state, done)
                    self.agent.replay()

                state = next_state

            p_mean,trade_time,win_time = np.append(p_mean,info['cur_revenue']),np.append(trade_time,info['trade_time']),np.append(win_time,info['trade_win'])
            values=[('FixedProfit',int(np.mean(p_mean))), ('TradeTimes',int(np.mean(trade_time))), ('TradeWin',int(np.mean(win_time)))]
            pb_i.add(1, values=values)
            with open(csv_path, 'a') as f:
                row = str(info['cur_revenue']) + ',' + str(info['trade_time']) + ',' + str(info['trade_win'])
                print(row, file=f)

        if self.mode == 'train':
            self._save()

    def _standard_scaler(self, env):
        states = []
        for _ in range(env.df_total_steps):
            action = np.random.choice(env.action_space)
            state, reward, done, info = env.step(action)
            states.append(state)
            if done:
                break      
        scaler = StandardScaler()
        scaler.fit(states)
        return scaler

    def _reward_clipping(self, val):
        result = 1 if val > 0 else 0 if val == 0 else -1
        return result

    def _load(self):
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'rb') as f:
            self.scaler = pickle.load(f)
        self.agent.load('{}/{}.h5'.format(self.mdl_dir, self.name))

    def _save(self):
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'wb') as f:
            pickle.dump(self.scaler, f)
        self.agent.save('{}/{}.h5'.format(self.mdl_dir, self.name))

In [ ]:
initial_money=1000000
commission = 0 if level == 1 else 0.002
episodes_times = 100
batch_size = 32
max_size = 500

env = Environment(df, initial_money = initial_money, mode = mode, commission = commission)
agent = Agent(max_size, batch_size)
main = Main(env, agent, mdl_dir, name, episodes_times, mode)
main.play_game()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                192       
                                                                 
 re_lu (ReLU)                (None, 48)                0         
                                                                 
 dense_1 (Dense)             (None, 96)                4704      
                                                                 
 re_lu_1 (ReLU)              (None, 96)                0         
                                                                 
 dense_2 (Dense)             (None, 153)               14841     
                                                                 
 reshape (Reshape)           (None, 3, 51)             0         
                                                                 
 softmax (Softmax)           (None, 3, 51)             0